# 有限域和多项式
本节快速的总结一下相关的有限域知识点 <br>
域有这样一个性质：在加法和乘法上具有封闭性。也就是说对域中的元素进行加法或乘法运算后的结果仍然是域中的元素。<br>

## 有限域$GF(p^n)$
我们最为关注的只有两种情况：n=1即$GF(p)$；p为2即$GF(2^n)$。前者称之为素域，后者称之为二进制域。在密码学中，有限域GF(p)是一个很重要的域，其中p为素数。简单来说，GF(p)就是 mod p，因为一个数 模p后，结果在[0, p-1]之间。对于元素a和b，那么(a+b) mod p和(a*b)mod p，其结果都是域中的元素。GF(p)里面的加法和乘法都是平时用的加法和乘法。GF(p)的加法和乘法单位元分别是0和1，元素的加法和乘法逆元都很容易理解和求得

$GF(p)$的空间是模p的完全剩余类$Z_p : \left\{0, 1, \cdots, p-1 \right\}$

举一个例子：$\mathbb{F}_{3221225473}$ ($3221225473 = 3 \cdot 2^{30} + 1$) 就是一个格罗瓦素域。

有限域的元素个数是一个素数的幂$p^n$，n为正整数，一般记为$GF(p^n)$

下面给一个引用自stark101中的一个FieldElement 实现 https://github.com/starkware-industries/stark101

In [1]:
class FieldElement:
    """
    Represents an element of F_(3 * 2**30 + 1).
    """
    k_modulus = 3 * 2**30 + 1
    generator_val = 5

    def __init__(self, val):
        self.val = val % FieldElement.k_modulus

    @staticmethod
    def zero():
        """
        Obtains the zero element of the field.
        """
        return FieldElement(0)

    @staticmethod
    def one():
        """
        Obtains the unit element of the field.
        """
        return FieldElement(1)

    def __repr__(self):
        # Choose the shorter representation between the positive and negative values of the element.
        return repr((self.val + self.k_modulus//2) % self.k_modulus - self.k_modulus//2)

    def __eq__(self, other):
        if isinstance(other, int):
            other = FieldElement(other)
        return isinstance(other, FieldElement) and self.val == other.val

    def __hash__(self):
        return hash(self.val)

    @staticmethod
    def generator():
        return FieldElement(FieldElement.generator_val)

    @staticmethod
    def typecast(other):
        if isinstance(other, int):
            return FieldElement(other)
        assert isinstance(other, FieldElement), f'Type mismatch: FieldElement and {type(other)}.'
        return other

    def __neg__(self):
        return self.zero() - self

    def __add__(self, other):
        try:
            other = FieldElement.typecast(other)
        except AssertionError:
            return NotImplemented
        return FieldElement((self.val + other.val) % FieldElement.k_modulus)

    __radd__ = __add__

    def __sub__(self, other):
        try:
            other = FieldElement.typecast(other)
        except AssertionError:
            return NotImplemented
        return FieldElement((self.val - other.val) % FieldElement.k_modulus)

    def __rsub__(self, other):
        return -(self - other)

    def __mul__(self, other):
        try:
            other = FieldElement.typecast(other)
        except AssertionError:
            return NotImplemented
        return FieldElement((self.val * other.val) % FieldElement.k_modulus)

    __rmul__ = __mul__

    def __truediv__(self, other):
        other = FieldElement.typecast(other)
        return self * other.inverse()

    def __pow__(self, n):
        assert n >= 0
        cur_pow = self
        res = FieldElement(1)
        while n > 0:
            if n % 2 != 0:
                res *= cur_pow
            n = n // 2
            cur_pow *= cur_pow
        return res

    def inverse(self):
        t, new_t = 0, 1
        r, new_r = FieldElement.k_modulus, self.val
        while new_r != 0:
            quotient = r // new_r
            t, new_t = new_t, (t - (quotient * new_t))
            r, new_r = new_r, r - quotient * new_r
        assert r == 1
        return FieldElement(t)

    def is_order(self, n):
        """
        Naively checks that the element is of order n by raising it to all powers up to n, checking
        that the element to the n-th power is the unit, but not so for any k<n.
        """
        assert n >= 1
        h = FieldElement(1)
        for _ in range(1, n):
            h *= self
            if h == FieldElement(1):
                return False
        return h * self == FieldElement(1)

    def _serialize_(self):
        return repr(self.val)

    @staticmethod
    def random_element(exclude_elements=[]):
        fe = FieldElement(randint(0, FieldElement.k_modulus - 1))
        while fe in exclude_elements:
            fe = FieldElement(randint(0, FieldElement.k_modulus - 1))
        return fe

In [2]:
FieldElement(1)

1

In [10]:
g=FieldElement.generator()*(3*2**20)
G=[g ** i for i in range(1024)]
print(g)
len(G)

15728640


1024

  𝔽×  表示  𝔽 的乘法循环群，其中排除了𝔽 的0元。𝔽× 有两个重要的性质，第一其大小为2^30，第二对于任意的i, 0≤𝑖≤30，存在一个子群，大小为2^i .

## 陪集coset

对上面的$G$ ，左乘一个 $\mathbb{F}^\times$ 的生成元, 就得到一个$G$ 的左陪集 [coset](https://en.wikipedia.org/wiki/Coset) <br>
陪集在后面的RScode编码中将会用到。

In [14]:
LC=[FieldElement.generator() * g for g in G]
LC[1]

78643200

## $GF(2^n)$

$GF(2^n)$中的的元素是系数为二进制0和1的多项式，最高不超过n-1次。一个元素可以被表示成一个长度为n的位矢量。例如二进制数$11001_2$在$GF(2^5)$中可以记作$x^4+x^3+1$


　　1.普通多项式运算。这个不必多说，从小学初中就开始学，就是我们认识的普通多项式。

　　2.系数在$Z_p$中的多项式运算。和普通多项式运算不同的是，系数要进行模p运算。模可以是任意素数，一般取二，是最简单的情况。

　　3.有限域$GF(2^n)$上的多项式运算。这种运算和计算机的运作方式很相似，对于一个有限域$GF(2^n)$我们定义如下要求：系数对2取模运算，最高次数小于n，多项式对n次素多项式取模运算。既然是域那就有逆元，可以用拓展欧几里得算法求逆。

　　下面分别介绍在$GF(2^n)$上的四则运算：

　　1.加法（减法）

　　$GF(2^n)$上的加法即比特串的异或运算，因为是异或，加减其实是一样的，都是异或而已。<br>
  
　　2.乘法

　　乘法即比特串的移位和异或运算。实际运算的时候和普通乘法很像，只不过加法变成了异或。下面的运算用了一个小技巧，异或的结果只和参与异或的1的个数有关，奇数则为1，偶数则为0。
　　或者可以表示为$011_2\times 101_2=(101_2)<<1\oplus (101_2)<<0=1111_2$其中<<为移位符。

　　3.取模

　　在$GF(2^n)$上取模运算就相当于除法。